In [2]:
import numpy as np
import cv2
from sklearn import svm
from sklearn.externals import joblib


C:\Users\acer\.conda\envs\opencv-env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
def load_img(f):
    f=open(f)
    lines=f.readlines()
    
    imgs,lab=[], []
    
    for i in range(len(lines)):
        
        fn,label = lines[i].split(' ')
        im1 = cv2.imread(fn)
        im1 = cv2.resize(im1,(256,256),interpolation=cv2.INTER_NEAREST)
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
        
        imgs.append(im1)
        lab.append(int(label))
        
    return imgs, lab

In [ ]:
train_x,train_y= load_img('train.txt')

In [ ]:
val_x,val_y = load_img('val.txt')

In [6]:
def calcSiftFeature(img):
    
    sift = cv2.xfeatures2d.SURF_create()
    
    keypoints, features = sift.detectAndCompute(img, None)
    return features


In [7]:
def learnVocabulary(features):
    wordCnt = 50
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.1)
    
    flags = cv2.KMEANS_RANDOM_CENTERS
    
    compactness, labels, centers = cv2.kmeans(features, wordCnt, None,criteria, 20, flags)
    return centers

In [8]:
def calcFeatVec(features, centers):
    featVec = np.zeros((1, 50))
    for i in range(0, features.shape[0]):
        
        fi = features[i]
        diffMat = np.tile(fi, (50, 1)) - centers
        
        sqSum = (diffMat**2).sum(axis=1)
        dist = sqSum**0.5
        
        sortedIndices = dist.argsort()
       
        idx = sortedIndices[0]
        
        featVec[0][idx] += 1
    return featVec

In [9]:
def build_center(train_x):
    features = np.float32([]).reshape(0, 64)
    for im in range(len(train_x)):
        print (im)
        
        img_f = calcSiftFeature(train_x[im])
        
        features = np.append(features, img_f, axis=0)
    
    centers = learnVocabulary(features)
    
    filename = "svm_centers.npy"
    np.save(filename, centers)
    print('詞袋之類啥的:',centers.shape)


In [ ]:
def cal_vec(train_x):
    centers = np.load("svm_centers.npy")
    data_vec = np.float32([]).reshape(0, 50)
    labels = np.float32([])
    
    for im in range(len(train_x)):
        
        
        img_f = calcSiftFeature(train_x[im])
        img_vec = calcFeatVec(img_f, centers)
        data_vec = np.append(data_vec,img_vec,axis=0)
        labels = np.append(labels,im)
    print('data_vec:',data_vec.shape)
    print('image features vector done!')
    return data_vec,labels

In [ ]:
def SVM_Train(train_x,train_y):
    
    clf = svm.SVC(decision_function_shape='ovo')
    
    clf.fit(train_x,train_y)
    joblib.dump(clf, "svm_model.m")

In [ ]:
def SVM_Test(val_x,val_y):
    
    clf = joblib.load("svm_model.m")
    
    res = clf.predict(val_x)
    num_test = val_x.shape[0]
    acc = 0
    for i in range(num_test):
        if val_y[i] == res[i]:
            acc = acc+1
    return acc/num_test,res

In [ ]:
build_center(train_x)

In [ ]:
data_vec,labels = cal_vec(train_x)

In [ ]:
SVM_Train(data_vec,train_y)


In [ ]:
print(train_x)
print(train_y)

In [ ]:
data_vec_val,labels_val = cal_vec(val_x)

In [ ]:
acc,res = SVM_Test(data_vec_val,val_y)
print(acc)
print(res)

In [ ]:
def predict(img):
    clf = joblib.load("svm_model.m")
    centers = np.load("svm_centers.npy")
    features = calcSiftFeature(img)
    featVec = calcFeatVec(features, centers)
    case = np.float32(featVec)
    res = clf.predict(case)
    return res

In [ ]:
test_x,test_y = load_img('test.txt')

In [ ]:
res=[]
for im in range(len(test_x)):
    res.append(predict(test_x[im]))

In [ ]:
data_vec_test,labels_test = cal_vec(test_x)

testacc,testres = SVM_Test(data_vec_test,test_y)
print(testacc)
print(testres)

In [ ]:
print (data_vec)